In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gc
import lightgbm as lgb
gc.collect()

4

In [9]:
sub_dir = "../submission2.csv"

In [10]:
df_bbox_submission = pd.read_csv(sub_dir)

In [11]:
len(df_bbox_submission)

99999

In [4]:
df_bbox_submission.head()

,ImageId,PredictionString
0,00000b4dcff7f799,/m/0dzf4 0.29 0.8515 0.29339 0.92297 0.44191 /...
1,00001a21632de752,/m/03q69 0.62 0.56335 0.58706 0.80463 1.0 /m/0...
2,0000d67245642c5f,/m/031n1 0.06 0.0 0.12647 0.71125 0.90622 /m/0...
3,0001244aa8ed3099,/m/035r7c 0.25 0.24612 0.06044 0.79299 0.43748...
4,000172d1dd1adce0,/m/04hgtk 0.12 0.29667 0.35318 0.31774 0.38799...


In [5]:
list_bboxes = []
for index, row in tqdm(df_bbox_submission.iterrows()):
    bboxes = []
    image_id = row["ImageId"]
    arrList = row["PredictionString"]
    predictList = arrList.split("/m/")[1:]
    for pred in predictList:
        predItem = pred.split(" ")
        clsId = "/m/"+predItem[0]
        confidence = float(predItem[1])
        XMin, YMin, XMax, YMax = [float(x) for x in predItem[2:6]]
        dict_item = {
            "LabelName":clsId,
            "Confidence":confidence,
            "XMin":XMin,
            "YMin":YMin,
            "XMax":XMax,
            "YMax":YMax,
            "ImageID":image_id,
            "IsGroupOf":0
        }
        list_bboxes.append(dict_item)

df_sub_fmt_bboxes = pd.DataFrame(list_bboxes)

In [6]:
df_sub_fmt_bboxes.to_csv("submission2_formatted.csv", index=False)

In [3]:
df_sub2_fmt = pd.read_csv("submission2_formatted.csv")

In [4]:
len(df_sub2_fmt)

97574101

In [ ]:
# !python oid_hierarchical_labels_expansion.py --json_hierarchy_file=../dataset/challenge-2019-label500-hierarchy.json --input_annotations=submission2_formatted.csv --output_annotations=submission2_formatted_levelled.csv --annotation_type=1

In [2]:
df_sub_to = pd.read_csv("submission2_formatted_levelled.csv")

In [3]:
len(df_sub_to)

132511810

In [4]:
df_sub_to.head()

,Confidence,ImageID,IsGroupOf,LabelName,XMax,XMin,YMax,YMin
0,0.29,00000b4dcff7f799,0,/m/0dzf4,0.92297,0.85150,0.44191,0.29339
1,0.27,00000b4dcff7f799,0,/m/04hgtk,0.80473,0.76490,0.44061,0.37286
2,0.26,00000b4dcff7f799,0,/m/04hgtk,0.63076,0.58915,0.50128,0.43388
3,0.25,00000b4dcff7f799,0,/m/035r7c,0.51779,0.47175,0.86208,0.60471
4,0.23,00000b4dcff7f799,0,/m/035r7c,0.93331,0.87764,0.79844,0.56654


In [5]:
df_grp = df_sub_to.groupby("ImageID")

In [6]:
out_file="submission_levelled.csv"

In [7]:
f = open(out_file,"w+")
results="ImageId,PredictionString\n"
f.write(results)
for image_id, grp in tqdm(df_grp):
    results=image_id+","
    for index, row in grp.iterrows():
        clsId = row["LabelName"]
        score = row["Confidence"]
        bbox = str(row["XMin"]) + " " + str(row["YMin"]) + " " + str(row["XMax"]) + " " + str(row["YMax"])
        results+= str(clsId) + " "+ str(score)+ " " + bbox + " "
    results +="\n"
    f.write(results)
f.close()

In [8]:
!gzip -c submission_levelled.csv > submission_levelled2.csv.gz